# Project 3: Classifier two-sample tests

```
From ML Theory to Practice
Universität Potsdam, fall semester 2025

Authors: Juan L. Gamella and Simon Bing
License: CC-BY-4.0 https://creativecommons.org/licenses/by/4.0/
```

## Setup

These packages should already be installed in your Python virtual environment.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

Set the torch random seed for reproducibility.

In [ ]:
torch.manual_seed(3)

### Download the image dataset

For this part we will use an existing dataset composed of images collected from the light tunnel.

First, choose the directory where it should be saved

In [ ]:
DOWNLOAD_DIR = './'

and download it

In [ ]:
from causalchamber.datasets import Dataset

# Download the dataset and store it, e.g., in the current directory
dataset = Dataset('lt_color_regression_v1', root=DOWNLOAD_DIR, download=True)

Let's visualize some images

In [ ]:
def plot_random_images(sample, n=5, seed=42):
    plt.figure(figsize=(n*2, 2))
    rng = np.random.default_rng(seed)
    for i,j in enumerate(rng.integers(len(sample), size=n)):
        plt.subplot(1,n,i+1)
        plt.imshow(sample[j])

In [ ]:
# Load the observations and images from an experiment (see experiment names below)
# Choose
images_ref = dataset.get_experiment(name='reference').as_image_array(size='100')
images_pol_45 = dataset.get_experiment(name='pol_1_45').as_image_array(size='100')
plot_random_images(images_ref)
plot_random_images(images_pol_45)

As you can see, it is quite hard to tell the difference at plain sight. If you look closer, you can notice the difference int the shape of the blur on the top of the image.

That's quite a subtle difference. Let's try to build a two-sample test that can pick up on it!

## Preparing the data for training

Before training, we need to do some preprocessing on our data so the training will be effective. In particular, we need to:

- Scale the images so the pixel intensities are in `[-1,1]` instead of `[0,255]`
- [One-hot encode](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics) our class labels (i.e., sample A or B)
- Transposes the images from `[height, width, color]` to `[color, height, width]`
- Load the images and labels as a Tensor so they can be used by Pytorch.

Pytorch uses the class `torch.utils.data.Dataset` to load data, and it is good practice to place all our preprocessing logic there.

**⇨ Task**

Complete the code below to write a class `TwoSampleDataset` that inherits from `torch.utils.data.Dataset` and carries out the pre-processing steps.

For the one-hot encoding, you can use

```python
pd.get_dummies(<class_labels>).values.astype(float)
```

In [ ]:
class TwoSampleDataset(torch.utils.data.Dataset):
    
    def __init__(self, images_0, images_1):
        """Take two image arrays, preprocess them and store the images and labels indicating to which array each image belongs.

        Pre-processing steps:
          - Scale the images so the pixel intensities are in `[-1,1]` instead of `[0,255]`
          - One-hot encode our class labels (i.e., sample A or B)
        """
            
        # TODO: your code goes here        

        self.images = None
        self.labels = None
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        """
        Returns data from the dataset as a tuple of (images, labels).

        Both must be tensors.
        """
        
        images = self.images[idx]
        labels = self.labels[idx]
        
        # Convert to tensors
        images = torch.from_numpy(image).float()
        labels = torch.from_numpy(labels).float()
        
        return images, labels

**⇨ Task**

Now, because we don't want to use all our images at once, write a simple function to sample random images from an image array. You can use the numpy random generator [`numpy.random.default_rng`](https://numpy.org/doc/stable/reference/random/generator.html) and its function [`choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html). Make sure you can set the seed for repeatability.

In [ ]:
def sample_array(n, image_array, seed=42):
    """
    Sample n images without replacement from the given image array, where images are ordered along the 0th axis.    
    """
    # TODO: your code goes here

**⇨ Task**

Test everything works. Take two random samples of size `n=100` of images from the experiments `reference` and `pol_1_45` above, and load them into the `TwoSampleDataset`.

In [ ]:
n = 100
# TODO: Your code here
# dataset = TwoSampleDataset(...)

# Check if everything works, e.g., call dataset[0]
# TODO: Your code here

## Building your classifier

The next step is to define the architecture of your neural classifier.

Let's with a simple [MLP](https://en.wikipedia.org/wiki/Multilayer_perceptron), consisting of

- an input layer with `30000` neurons, i.e., the size of a flattened image
- some hidden layers
- output layer of size 2, i.e., the logits indicating the classification made by the model

Use the ReLU activation function, e.g., `torch.nn.ReLU()`.

**⇨ Task**

Complete the code below to create a model like above. If you need to, you can use [this tutorial](https://docs.pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) for reference.

Use a parameter `hidden_layers` to parametrize the number and size of hidden layers of the model, e.g., `hidden_layers = [10]` should result in a model with one hidden layers of 10 neurons, while `hidden_layers = [10,10,10]` should result in a model with three such layers stacked one after another. This will make your life easier later as we explore the effects of model capacity.

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, hidden_layers = [], device="cpu"):
        super().__init__()
        
        # Construct layers
        # First layer is just a flattening operation
        self.flatten = nn.Flatten()

        # Build the input, hidden and output layers
        # TODO: your code here

        # Store the model in the correct device (by default `cpu`)
        self.device = device
        self.to(device)

    def forward(self, x):
        # The forward pass of the model
        # x is the input
        # should return the values of the two neurons in the last layer (logits)
        # TODO: your code goes here
        pass

## Training the classifier

Now we need to write the training loop that will fit the classifier to our images.

There are some things we need to do.

**⇨ Task**

First, write a function `split_dataset(dataset, ratios)` which takes in a dataset and splits it at random into smaller disjoint datasets following the given ratios. You can follow the example for `random_split` shown [here](https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.random_split). Make sure you set a random seed for repeatability.

In [ ]:
# TODO: your code here

### The training loop

Now it's time to write the training loop for our model. It needs several ingredients

#### The model

e.g., an instance of your `SimpleMLP` class. For now, let's start with one hidden layer with 10 neurons

In [ ]:
model = SimpleMLP(hidden_layers = [10])

#### The loss function

We will use the [cross-entropy loss](https://en.wikipedia.org/wiki/Cross-entropy) (a.k.a. the KL divergence) to compare the ground-truth and the class distribution produced by your model. Fortunately, it is already nicely implemented in [`torch.nn.CrossEntropyLoss()`](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) to directly manage logits (the output of our model).

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

#### The optimizer

We will use the ADAM optimizer. Set a learning rate of `1e-3` and a weight decay of `1e-5`.

In [ ]:
optimizer = torch.optim.Adam
opt_args = {'lr': 1e-3, 'weight_decay': 1e-5}

#### The data

We will need a training dataset and a separate test dataset to monitor the training. We can also set aside a validation dataset to use only once the model has been trained.

**⇨ Task**

As before, take two random samples of size `n=500` of images from the experiments `reference` and `pol_1_45` above, load them into a `TwoSampleDataset` and split them into training, test and validation datasets with `ratios=[10, 90, 900]`, i.e., we want only 10 images for training and a large validation set.

In [ ]:
n = 500
# TODO: your code goes here

**⇨ Task**

Now write the training loop for your model. You can follow the steps in [this tutorial](https://docs.pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) and adjust it to your code.

As you train, remember to store the training and test loss after each batch in `train_losses` and `test_losses` for later plotting and diagnostics.

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
# TODO: your code goes here

### Training and diagnostics

**⇨ Task**

Now, train your model for 100 epochs with a batch size of 10, and plot the training and test losses.

In [ ]:
model = SimpleMLP(hidden_layers = [10])
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam
opt_args = {'lr': 1e-3, 'weight_decay': 1e-5}

# TODO: your code goes here

[❓] **Question** Looking at your train/test loss curves, would you say your model has trained correctly? Why / why not?

**⇨ Task**
  
Now, train and evaluate two more models:

- A flat model (`hidden_layers = []`), i.e., a linear transformation of the inputs.
- A more complex model, with three hidden layers of 64 neurons, i.e., `hidden_layers = [64, 64, 64]`

As before, plot the curves of the training and test loss for each model.

In [ ]:
# TODO: your code goes here

**⇨ Task**

Now, train each of the three models 10 times, and plot the resulting loss curves.

The curves should look different for each training run - make sure you're setting the random seed correctly!

In [1]:
# TODO: your code goes here

[❓] **Question** Do you observe any difference in the training of each model?

[❓] **Question** Are any models overfitting to the training data? Which model do you think generalizes best? Explain your reasoning / intuition.

## Transforming our classifier into a Two-Sample Test

Now comes the fun part, as we transform our classifier into a two-sample test.

Given a test-set $\mathcal{D}_{te} = \{(z_1, l_1), \ldots (z_{n_{te}}, l_{n_{te}})\}$, recall that the test statistic (eq. (2) in the paper) is given by

$$\hat{t} := \frac{1}{n_{te}} \sum_{(z_i, l_i) \in \mathcal{D}_{te}} \mathbb{I}\left [ \mathbb{I} \left( f(z_i) > \frac{1}{2}\right) = l_i\right],$$

where $f$ is a classifier and $\mathbb{I}$ is the [indicator function](https://en.wikipedia.org/wiki/Indicator_function).

Let's build all these ingredients.

#### The softmax function

Above, $f$ is a function $f: \mathcal{X} \to [0,1]$ that denotes the estimated probability that $X \in \mathcal{X}$ belongs to class 0. However, our model returns two real numbers (logits) as its prediction.

To transform them into probabilities, we can use the [softmax function](https://en.wikipedia.org/wiki/Softmax_function): given logits $\boldsymbol{z} = (z_1, \ldots, z_K) \in \mathbb{R}^K$, it returns a vector $\sigma(\boldsymbol{z}) \in [0,1]^K$ where each component is given by

$$\sigma(\boldsymbol{z})_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}.$$

**⇨ Task**

Implement the softmax function and perform the simple checks below.

In [ ]:
def softmax(x):
    # TODO: your code goes here
    return None

Some basic checks:

In [ ]:
assert (softmax([1,1]) == [0.5,0.5]).all() # Softmax of n same elements is [1/n, ..., 1/n]
assert (softmax([0,-np.inf]) == [1,0]).all() # Softmax of [0, -infinity] = [1,0]

**⇨ Task**

Now, write a function to compute the accuracy of a model on a given dataset. To transform a tensor `t` (e.g., the output of the model) to a numpy array, you can use

```python
t.cpu().detach().numpy()
```

In [ ]:
def compute_accuracy(mode, dataset):
    # TODO: your code goes here
    return None

Now we will evaluate the accuracy of the three models (base model, flat model and complex model) on the left-out validation dataset..

[❓] **Question** Which model do you think will achieve higher accuracy? Make a guess and explain your intuition or reasoning.

**⇨ Task**

Now, evaluate the accuracy of three models you trained above on the validation dataset.

In [ ]:
# TODO: your code goes here

[❓] **Question** Was your guess correct? If not, what do you think is going on?

### Computing a p-value

Now we will write the code to compute a p-value given the accuracy produced by the model.

[❓] **Question** Given the accuracy of the three models, what do you expect their p-values to be?

[❓] **Question** Under the null hypothesis and small $n_{te}$ what distribution does the number of correct labels $n_{te}\hat{t}$ follow? Hint: see section 3.1 of the [paper](https://arxiv.org/pdf/1610.06545).

**⇨ Task**

Write a function to compute the cumulative distribution function $F$ of this distribution. You can check its Wikipedia page for the formula.

> Hint: you can just write a wrapper around the `.cdf` function provided by [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html#discrete-distributions) for the correct distribution (see [here](https://docs.scipy.org/doc/scipy/reference/stats.html#discrete-distributions)).

In [ ]:
def cdf(t, n):
    # TODO: your code goes here
    pass

You can check your work with these simple tests (NOTE: failure means you made a mistake, no failed tests doesn't mean you didn't)

In [ ]:
assert cdf(0, 1) == 0.5
assert cdf(1, 1) == 1
assert cdf(0, 2) == 0.25
assert cdf(1, 2) == 0.75
assert cdf(2, 2) == 1

Remember that the p-value is the probability, under the null hypothesis, of observing a number of correctly classified observations more extreme than the one obtained. That is, given an accuracy $\hat{t}$ in a test set of size $n_{te}$, the probability is

$$P(X \geq k \cup X \leq n_{te}-k) \text{ if } \hat{t} > \frac{1}{2}, \text{ and } P(X \leq k \cup X \geq n_{te}-k) \text{ otherwise},$$

where $k := \hat{t}n_{te}$.



[❓] **Question** Rewrite the above expression in terms of the cumulative distribution function (CDF) $F$ that you computed above.

**⇨ Task**

Based on your derivation, write a function that takes in the accuracy of the model on a dataset, the size of the dataset, and returns the p-value under the null hypothesis.

In [ ]:
def compute_pvalue(accuracy, n):
    # TODO: Your code goes here
    pass

Again, here are some checks for your code

In [ ]:
assert compute_pvalue(0.5, 100) == 1
assert compute_pvalue(0.5, 10) == 1
assert compute_pvalue(0.5, 2) == 1
assert compute_pvalue(1, 100) == compute_pvalue(0, 100)
assert np.isclose(compute_pvalue(0, 100), 0)
assert np.isclose(compute_pvalue(1, 100), 0)

**⇨ Task**

Now, compute the the p-values for the three models on the validation dataset that left out above.

In [ ]:
for name, model in zip(['base', 'flat', 'complex'], [trained_model, flat_model, complex_model]):
    accuracy = compute_accuracy(model, dataset_validate)
    pval = compute_pvalue(accuracy, len(dataset_validate))
    print(name)    
    print('  accuracy', accuracy)
    print('  pvalue', pval)

[❓] **Question** Did the p-values match your guess above?

### Putting it all together

Now let's put it all together into a single function that implements our classifier two-sample test.


The function takes in the two samples we want to test and

1. Combines them into a Pytorch dataset
2. Splits the dataset into train and test
3. Trains a classifier
4. Computes the accuracy of the classifier on the test set
5. Returns the accuracy statistic and the corresponding p-value

**⇨ Task**

Complete the code below. Make sure that you can ensure repeatability with the `seed` parameter.

For diagnosing, also return the trained model and the loss curves observed during training.

In [ ]:
def c2st(sample_a, sample_b, model, ratios, seed=42):
    # Combine sample_a and sample_b into TwoSampleDataset
    # TODO: your code goes here
    
    # Split into train / test datasets (use ratios and seed)    
    # TODO: your code goes here

    # Train your model
    # TODO: your code here

    # Compute the accuracy and p-value on the test set
    # TODO: your code here
    
    # Return the p-value, accuracy statistic and the trained model
    pass

**⇨ Task**

Run the code below to check that everything is working as you expect.

In [ ]:
sample_a = sample_array(50, images_ref)
sample_b = sample_array(50, images_pol_45)

torch.manual_seed(1)
model = SimpleMLP(hidden_layers = [10])

pval,accuracy,_,train_losses,test_losses = c2st(sample_a, sample_b, model, [10,90], seed=42)

print(f"p-value = {pval} (accuracy = {accuracy:.3f})")
plt.figure()
plt.plot(train_losses, label="Train loss")
plt.plot(test_losses, label="Test loss")
plt.title('Flat model')
plt.legend()

## Evaluating the level of the test

Let's evaluate the level of our test and how it is affected by model complexity.

**⇨ Task**

Let's begin by evaluating the level of our test.

Split the `reference` dataset into 200 samples with `n=50` observations each.

Then, write a function that takes in a model and runs the 2CST on 100 pairs of these samples, storing the accuracy, pvalues and training / test loss curves. Use `10` observations for the training data and `90` for the testing data.

> Hint: remember to instantiate a new model for each of the 100 repetitions, or you will be training the same model over and over again!

In [ ]:
# TODO: your code goes here

[❓] **Question** What do you expect the distributions of accuracies and p-values to look like?

**⇨ Task**

Using [`sns.histplot`](https://seaborn.pydata.org/generated/seaborn.histplot.html), plot the distributions of the accuracies and pvalues for each model.

In [ ]:
# TODO: your code goes here

[❓] **Question** Did the results match your expectations? Why or why not?

[❓] **Question** Will all tests have correct level? Make a guess and explain your intuition.

**⇨ Task**

Using the stored p-values, make a plot with the nominal level $\alpha$ on the x-axis, and the actual rejection rate on the y-axis. Combine the results for all models into a single plot. For reference, plot the $y=x$ line in the range `[0,1]`.

In [ ]:
alphas = np.linspace(0, 1, 100)
# TODO: your code goes here

[❓] **Question** Do the tests appear valid at all levels $\alpha$?

[❓] **Question** Do the tests appear well *calibrated* at all levels $\alpha$? Make a guess and explain your intuition.

[❓] **Question** Does the model complexity affect the *validity* of the test? Why or why not? Explain your reasoning and/or intuition.

[❓] **Question** Does the model complexity affect the calibration of the test? Why or why not? Explain your reasoning and/or intuition.

[❓] **Question** What if a model does not train correctly? Is the test still valid at level $\alpha$?

[❓] **Question** Will any classifier $f: \mathcal{X} \to [0,1]$ also result in a valid test?

## Evaluating the power of the test

Now we will evaluate the power of our test and how it is affected by model complexity.

As in the previous project, we will evaluate power by see how each test rejects the null hypothesis when it is not true (i.e., images from different experiments).

[❓] **Question** Which test do you expect to be more powerful? Explain your reasoning / intuition.

**⇨ Task**

Adapt the code used for our level experiments above. This time, take 100 samples of size `n=50` from the `reference` experiment, and 100 samples from the `pol_1_45` experiment. You can use the function `sample_array` that you wrote before.

Then, write a function that takes in a model and runs the 2CST on the 100 pairs of these samples, storing the accuracy and pvalues and training/test curves. Use `10` observations for the training data and `90` for the testing data.

In [ ]:
# TODO: your code goes here

**⇨ Task**

As before, plot the distributions of the accuracies and p-values for each model using [`sns.histplot`](https://seaborn.pydata.org/generated/seaborn.histplot.html).

In [ ]:
# TODO: your code goes here

[❓] **Question** From the above plots, which test is the most powerful? Which is the least? Explain your reasoning / intuition.

[❓] **Question** From the above plots, can you already tell which test is the most powerful? Which is the least? Explain your reasoning / intuition.

**⇨ Task**

Now, using the stored p-values, calculate how often each test rejects the null hypothesis for different levels $\alpha$.

Plot your results in a plot with $\alpha$ on the x-axis and the actual rejection rate on the y-axis.

In [ ]:
alphas = np.linspace(0, 1, 1000)
# TODO: your code goes here

[❓] **Question** Which test is more powerful? Did your prediction hold?

**⇨ Task**

Now, repeat the above experiment & analysis, but this time increase the training sample size to 100.

As before, plot the distributions of p-values and the rejection rate of each test.

In [ ]:
# TODO: your code goes here

[❓] **Question** Which test is now the most powerful? If there has been a change, can you provide intuition as to why this happened?

**⇨ Task**

Plot the training / test loss curves resulting from the training.

In [ ]:
# TODO: your code goes here

[❓] **Question** Do the loss curves support your hypothesis? Why / why not?